In [29]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
from time import time
xgb.__version__

'1.5.0'

In [2]:
import pandas as pd


X_train = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/X_train.csv', header=None).values
y_train = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/y_train.csv', header=None).values.ravel().astype(int)

X_test = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/X_test.csv', header=None).values
y_test = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/y_test.csv', header=None).values.ravel().astype(int)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (9119, 16)
y_train.shape: (9119,)
X_test.shape: (4492, 16)
y_test.shape: (4492,)


In [3]:
X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid/Test sizes:', y_train_sub.shape[0], y_valid.shape[0], y_test.shape[0])

Train/Valid/Test sizes: 7295 1824 4492


In [4]:
y_train_sub

array([3, 0, 3, ..., 3, 4, 2])

In [12]:
y_test

array([2, 0, 5, ..., 4, 6, 3])

In [5]:
np.unique(y_train_sub)

array([0, 1, 2, 3, 4, 5, 6])

In [6]:
dtrain = xgb.DMatrix(X_train_sub, label=y_train_sub)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

In [7]:
num_round = 1000

In [16]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softmax']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['mlogloss']),
        'num_class': trial.suggest_categorical('num_class', [7])

    }

    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid)

    acc = accuracy_score(y_valid, predictions)
    
    return acc

In [17]:
study = optuna.create_study(direction='maximize')

[I 2021-11-16 15:17:57,865] A new study created in memory with name: no-name-863f7508-4374-469d-9b93-9adec112af9d


In [18]:
%%time
study.optimize(objective, n_trials=2)

[I 2021-11-16 15:18:02,245] Trial 0 finished with value: 0.9139254385964912 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.08959278301488241, 'alpha': 0.034589419521844654, 'colsample_bytree': 0.9658241624365731, 'subsample': 0.5808919143361457, 'learning_rate': 0.009581421719647683, 'max_depth': 9, 'min_child_weight': 67, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 0 with value: 0.9139254385964912.
[I 2021-11-16 15:18:09,675] Trial 1 finished with value: 0.8338815789473685 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.0261098832308896, 'alpha': 0.15112562041527816, 'colsample_bytree': 0.7836143885528984, 'subsample': 0.5217363054336831, 'learning_rate': 0.008015129316045467, 'max_depth': 17, 'min_child_weight': 259, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 0 with value: 0.9139254385964912.


CPU times: user 11.4 s, sys: 80.6 ms, total: 11.4 s
Wall time: 11 s


In [19]:
%%time
study.optimize(objective, n_trials=100)

[I 2021-11-16 15:18:32,184] Trial 2 finished with value: 0.8766447368421053 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 2.1040184452366053, 'alpha': 0.005116629399450099, 'colsample_bytree': 0.7660548418045704, 'subsample': 0.9654036457617079, 'learning_rate': 0.04924854372979283, 'max_depth': 3, 'min_child_weight': 172, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 0 with value: 0.9139254385964912.
[I 2021-11-16 15:19:22,122] Trial 3 finished with value: 0.9237938596491229 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.02159971932830458, 'alpha': 0.3819006809371373, 'colsample_bytree': 0.6552831234656763, 'subsample': 0.9056665331256465, 'learning_rate': 0.09984134446658842, 'max_depth': 20, 'min_child_weight': 43, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 3 with value: 0.9237938596491229.
[I 2021-11-16 15:19:24,793] Trial 4 finished with value: 0.8640350877192983 and parameters:

CPU times: user 27min 33s, sys: 6.01 s, total: 27min 39s
Wall time: 27min 2s


In [20]:
study.best_trial.params

{'objective': 'multi:softmax',
 'tree_method': 'gpu_hist',
 'lambda': 0.019613630447437882,
 'alpha': 0.2766220032240862,
 'colsample_bytree': 0.7194811956259057,
 'subsample': 0.9724036039411196,
 'learning_rate': 0.0019627396144725937,
 'max_depth': 9,
 'min_child_weight': 1,
 'eval_metric': 'mlogloss',
 'num_class': 7}

In [21]:
%%time
study.optimize(objective, n_trials=100)

[I 2021-11-16 16:17:13,598] Trial 102 finished with value: 0.930921052631579 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.04143089538227547, 'alpha': 0.13050157947426363, 'colsample_bytree': 0.6984381233460304, 'subsample': 0.7628713966213765, 'learning_rate': 0.0017725908739943043, 'max_depth': 9, 'min_child_weight': 2, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 61 with value: 0.9331140350877193.
[I 2021-11-16 16:17:23,914] Trial 103 finished with value: 0.9205043859649122 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.043293296474767574, 'alpha': 0.13380389065399795, 'colsample_bytree': 0.6953810442540308, 'subsample': 0.7620062716580102, 'learning_rate': 0.0018368078442897683, 'max_depth': 9, 'min_child_weight': 14, 'eval_metric': 'mlogloss', 'num_class': 7}. Best is trial 61 with value: 0.9331140350877193.
[I 2021-11-16 16:17:30,859] Trial 104 finished with value: 0.9155701754385965 and p

CPU times: user 27min 45s, sys: 5.81 s, total: 27min 51s
Wall time: 27min 5s


In [22]:
study.best_trial.params

{'objective': 'multi:softmax',
 'tree_method': 'gpu_hist',
 'lambda': 0.019613630447437882,
 'alpha': 0.2766220032240862,
 'colsample_bytree': 0.7194811956259057,
 'subsample': 0.9724036039411196,
 'learning_rate': 0.0019627396144725937,
 'max_depth': 9,
 'min_child_weight': 1,
 'eval_metric': 'mlogloss',
 'num_class': 7}

In [51]:
params = {'objective': 'multi:softmax',
 'tree_method': 'gpu_hist',
 'lambda': 0.019613630447437882,
 'alpha': 0.2766220032240862,
 'colsample_bytree': 0.7194811956259057,
 'subsample': 0.9724036039411196,
 'learning_rate': 0.0019627396144725937,
 'max_depth': 9,
 'min_child_weight': 1,
 'eval_metric': 'mlogloss',
 'num_class': 7}

In [25]:
xgtest = xgb.DMatrix(X_test) 

In [52]:
dtrain = xgb.DMatrix(X_train_sub, label=y_train_sub)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

model = xgb.train(params, dtrain, num_round)
predictions = model.predict(dvalid)

acc = accuracy_score(y_valid, predictions)
print(acc)

0.9331140350877193


In [53]:
test_preds = model.predict(xgtest)

acc = accuracy_score(y_test, test_preds)
print(acc)

0.9205253784505788


In [47]:
%%time
start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=2021)
preds = 0
i=1
for train_index, val_index in kf.split(X_train,y_train):
    #if i < 4:
        print(i)
        xgtrain, xgval = X_train[train_index], X_train[val_index]
        ygtrain, ygval = y_train[train_index], y_train[val_index]
        xgtrain = xgb.DMatrix(xgtrain, label = ygtrain ) 
        xgval = xgb.DMatrix(xgval, label = ygval ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=100000, evals=watchlist, 
                verbose_eval=100, 
                early_stopping_rounds = 300)
        
        preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #train_oof[val_index] = ypred
        #test_preds += preds/5
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_fold_'+str(i)+'.model')
        #del clf
        #gc.collect()
        i +=1
        print('Fold accuracy:', accuracy_score(ygval, ypred))
        
        break

1
[0]	train-mlogloss:1.94559	eval-mlogloss:1.94561
[100]	train-mlogloss:1.91466	eval-mlogloss:1.91632
[200]	train-mlogloss:1.88471	eval-mlogloss:1.88801
[300]	train-mlogloss:1.85564	eval-mlogloss:1.86059
[400]	train-mlogloss:1.82740	eval-mlogloss:1.83397
[500]	train-mlogloss:1.80003	eval-mlogloss:1.80816
[600]	train-mlogloss:1.77341	eval-mlogloss:1.78307
[700]	train-mlogloss:1.74755	eval-mlogloss:1.75870
[800]	train-mlogloss:1.72239	eval-mlogloss:1.73501
[900]	train-mlogloss:1.69788	eval-mlogloss:1.71190
[1000]	train-mlogloss:1.67402	eval-mlogloss:1.68943
[1100]	train-mlogloss:1.65076	eval-mlogloss:1.66754
[1200]	train-mlogloss:1.62806	eval-mlogloss:1.64617
[1300]	train-mlogloss:1.60593	eval-mlogloss:1.62534
[1400]	train-mlogloss:1.58432	eval-mlogloss:1.60500
[1500]	train-mlogloss:1.56322	eval-mlogloss:1.58514
[1600]	train-mlogloss:1.54263	eval-mlogloss:1.56578
[1700]	train-mlogloss:1.52248	eval-mlogloss:1.54686
[1800]	train-mlogloss:1.50279	eval-mlogloss:1.52838
[1900]	train-mlogloss:

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 1501.1480989456177
Fold accuracy: 0.9226973684210527
CPU times: user 25min 49s, sys: 20.6 s, total: 26min 10s
Wall time: 25min 1s


In [48]:
print('Fold accuracy:', accuracy_score(ygval, ypred))

Fold accuracy: 0.9226973684210527


In [49]:
accuracy_score(y_test, preds)

0.9216384683882458